In [120]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [121]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [122]:
news_df = pd.DataFrame(
    data=None,
    index=None,
    columns=['date','title','link','content']
)

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [123]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [124]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)

    
    print(news_urls)
    return news_urls

In [125]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls):
    news_titles=[]
    news_contents=[]
    news_dates=[]

    for url in news_urls:
        news_html = get_soup_obj(url)


        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

 
        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)

    a = pd.DataFrame({'date':news_dates, 'title':news_titles, 'link':news_urls, 'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [126]:
def clearnews():
    special_char = '\/:*?"<>|\''
    for clear in special_char:
        if clear in title:
            print(title.find(clear),clear)
            title = title.replace(clear,'')
    print(title)

def clearnews():
    special_char = '\/:*?"<>|\'\''
    for clear in special_char:
        if clear in content:
            print(content.find(clear),clear)
            content = content.replace(clear,'')
    print(content)

In [127]:
# while True:
news_urls = get_top3_news_info()
a = F_crawling(news_urls)
news_df=pd.concat([news_df,a])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print(news_df.head())

# time.sleep(50)

['https://n.news.naver.com/mnews/article/003/0011957925?sid=101', 'https://n.news.naver.com/mnews/article/009/0005154753?sid=101', 'https://n.news.naver.com/mnews/article/031/0000757038?sid=101']
                  date                                      title   
0  2023-07-06 14:59:13       경계현 "삼성 HBM 점유율 여전히 50% 이상"…시장 우려 일축  \
1  2023-07-06 14:59:05  “해지하러 은행 간다 vs 각서 써주겠다”…새마을금고 앞서 벌어진 설왕설래   
2  2023-07-06 14:59:03                  현대백화점그룹, 단일 지주회사 체제 구축 추진   

                                                link   
0  https://n.news.naver.com/mnews/article/003/001...  \
1  https://n.news.naver.com/mnews/article/009/000...   
2  https://n.news.naver.com/mnews/article/031/000...   

                                             content  
0  [\n임직원 대상 '위톡'서 밝혀…"고객사 평가 긍정적"삼성전자 HBM3·DDR5 ...  
1  [\n\n\n\n\n 6일 오전 서울 종로구의 한 새마을금고 지점에 관련 안내문이 ...  
2  [\n현대지에프홀딩스, 6일 이사회 열고 현대백화점 현물출자 진행키로현대백화점그룹이...  


In [128]:
NewsDate = news_df['date'].to_list()
NewsTItle = news_df['title'].to_list()
NewsUrl = news_df['link'].to_list()
NewsContent = news_df['content'].to_list()

In [129]:
for date,title,url,content in zip(NewsDate, NewsTItle, NewsUrl, NewsContent):
    sql = "INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES (%s, %s, %s, %s)" % ("'"+date+"'", "'"+title+"'", "'"+url+"'", "'"+content+"'")
    print(sql)
    cur.execute(sql)
conn.commit()
conn.close()

INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES ('2023-07-06 14:59:13', '경계현 "삼성 HBM 점유율 여전히 50% 이상"…시장 우려 일축', 'https://n.news.naver.com/mnews/article/003/0011957925?sid=101', '[
임직원 대상 '위톡'서 밝혀…"고객사 평가 긍정적"삼성전자 HBM3·DDR5 시대에도 '초격차' 지속 다짐



[용인=뉴시스] 김종택 기자 = 27일 오후 경기도 용인시 삼성전자 기흥캠퍼스에서 열린 ‘용인 첨단 시스템반도체 국가산단 성공을 위한 제3차 범정부 추진지원단 회의’에서 협약식 후 경계현 삼성전자 대표이사가 발언하고 있다. (공동취재사진) 2023.06.27. photo@newsis.com[서울=뉴시스]이인준 기자 = 경계현 삼성전자 디바이스솔루션(DS) 부문장(사장)이 인공지능(AI) 반도체 시대의 총아인 고대역폭메모리(HBM) 제품 시장에 대한 강한 자신감을 나타냈다.6일 업계에 따르면 경 사장은 전날 사내 소통 채널 '위톡'에서 "삼성 고대역폭메모리(HBM) 제품의 시장 점유율이 여전히 50% 이상"이라고 말했다.HBM은 여러 개의 D램을 수직으로 연결해 기존 D램보다 데이터 처리 속도를 끌어올린 제품이다. 데이터 처리량이 급격하게 증가하는 AI용 반도체를 개발하는데 필수적인 요소로 부상하고 있다. 일각에서는 최근 미국 반도체 기업인 엔비디아와 SK하이닉스의 4세대 'HBM3' 협력 강화로 삼성전자의 시장 지배력이 흔들리는 것이 아니냐는 우려가 나오자 경 사장이 나서 일축한 것이다. HBM 시장은 아직 시장 형성 초기 단계로, 전체 D램 시장에서 차지하는 비중은 미미하다. 하지만 AI 개발 열풍 속에서 가파른 성장세를 나타내고 있다. 시장조사업체 트렌드포드에 따르면 전 세계 HBM 수요는 올해 2억9000만 기가바이트(GB)로 작년보다 60% 가까이 증가하고 내년에는 30% 더 성장

ProgrammingError: (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'위톡\'서 밝혀…"고객사 평가 긍정적"삼성전자 HBM3·DDR5 시...\' at line 1')

In [ ]:
# ###데이터 프레임으로 만들기###
# from datetime import date, timedelta
# import pandas as pd

# #데이터 프레임 만들기

# # list=[]
# a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

# # list.append([news_df,a])

# #데이터 프레임 저장
# now = datetime.now() 
# a.to_csv('C:\\big16\\trading_project\\ssuyan\\REAL_NEWS\\_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)
